In [0]:
from pyspark.sql.functions import col, sum
spark.conf.set("spark.sql.join.preferSortMergeJoin", "false")

In [0]:
customers_df = spark.read.table("hive_metastore.default.fake_customers")
orders_df = spark.read.table("hive_metastore.default.fake_orders")
zipcodes_df = spark.read.csv("/Volumes/mp_spark_deep_ddve/default/zipcode_data/zipcode_data.csv", header=True, inferSchema=True)

In [0]:
joined_df = orders_df.join(customers_df.hint("SHUFFLE_HASH"), orders_df.customer_id == customers_df.customer_id, how="left")

In [0]:
agg_zipcodes_df = zipcodes_df.groupby("zipCode").agg(sum("population").alias("total_population"))
joined_zipcodes_df = joined_df.join(agg_zipcodes_df.hint("SHUFFLE_HASH"), joined_df.zip_code == agg_zipcodes_df.zipCode, how="left")

In [0]:
joined_zipcodes_df.show(1000)

+------------+-------------------+-----------+------------+----------------+--------------+------------+----------+--------+----------+-----------+--------------------+--------------------+--------------------+------------+--------------------+-----+--------+--------------------+-------+----------------+
|    order_id|    order_timestamp|customer_id|order_amount|product_category|payment_method|order_status|product_id|quantity|unit_price|customer_id|                name|               email|        addressLine1|addressLine2|                city|state|zip_code|        phone_number|zipCode|total_population|
+------------+-------------------+-----------+------------+----------------+--------------+------------+----------+--------+----------+-----------+--------------------+--------------------+--------------------+------------+--------------------+-----+--------+--------------------+-------+----------------+
|842066461547|2023-06-02 00:00:00|       7026|       992.0|            Home| Wire 

In [0]:
joined_zipcodes_df.explain()

== Physical Plan ==
AdaptiveSparkPlan isFinalPlan=false
+- == Initial Plan ==
   ShuffledHashJoin [cast(zip_code#125 as int)], [zipCode#184], LeftOuter, BuildRight
   :- Exchange hashpartitioning(cast(zip_code#125 as int), 200), ENSURE_REQUIREMENTS, [plan_id=594]
   :  +- ShuffledHashJoin [cast(customer_id#148 as bigint)], [customer_id#118L], LeftOuter, BuildRight
   :     :- Exchange hashpartitioning(cast(customer_id#148 as bigint), 200), ENSURE_REQUIREMENTS, [plan_id=587]
   :     :  +- Project [order_id#146L, order_timestamp#147, customer_id#148, order_amount#149, product_category#150, payment_method#151, order_status#152, product_id#153, quantity#154, unit_price#155]
   :     :     +- Filter if (isnotnull(_databricks_internal_edge_computed_column_skip_row#741)) (_databricks_internal_edge_computed_column_skip_row#741 = false) else isnotnull(raise_error(DELTA_SKIP_ROW_COLUMN_NOT_FILLED, map(keys: [], values: []), NullType))
   :     :        +- FileScan parquet hive_metastore.default